# Download the Dataset

In [1]:
!wget https://philharmonia-assets.s3-eu-west-1.amazonaws.com/uploads/2020/02/12112005/all-samples.zip

--2020-06-06 10:30:53--  https://philharmonia-assets.s3-eu-west-1.amazonaws.com/uploads/2020/02/12112005/all-samples.zip
Resolving philharmonia-assets.s3-eu-west-1.amazonaws.com (philharmonia-assets.s3-eu-west-1.amazonaws.com)... 52.218.60.16
Connecting to philharmonia-assets.s3-eu-west-1.amazonaws.com (philharmonia-assets.s3-eu-west-1.amazonaws.com)|52.218.60.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 260849422 (249M) [application/zip]
Saving to: ‘all-samples.zip’

all-samples.zip     100%[===================>] 248.76M  30.5MB/s    in 8.8s    

2020-06-06 10:31:03 (28.2 MB/s) - ‘all-samples.zip’ saved [260849422/260849422]



# Extract zip files

In [0]:
from zipfile import ZipFile
with ZipFile('all-samples.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [0]:
import os
dir_path='./all-samples'

In [0]:
for path, dir_list, file_list in os.walk(dir_path):
    for file_name in file_list:
        if file_name.endswith(".zip"):
            abs_file_path = os.path.join(path, file_name)

            parent_path = os.path.split(abs_file_path)[0]
            output_folder_name = os.path.splitext(abs_file_path)[0]
            output_path = os.path.join(parent_path, output_folder_name)

            zip_obj = ZipFile(abs_file_path, 'r')
            zip_obj.extractall(output_path)
            zip_obj.close()

# Start Trials

In [0]:
path_dataset='./all-samples/all-samples/'

In [0]:
instruments = ['banjo','bass-clarinet','bassoon', 'cello','clarinet','contrabassoon',
               'english-horn','double-bass','flute','french-horn','guitar','mandolin',
               'oboe','percussion','saxophone','trombone','trumpet','tuba','viola','violin']
classes = []

for root, dirnames, filenames in os.walk(path_dataset):
    for filename in filenames:
        for instrument in instruments:
            if instrument in filename:
                classes.append(instrument)

In [10]:
import pandas as pd
ds_Classes = pd.Series(classes)
ds_Classes.value_counts()

clarinet         1790
violin           1502
bassoon          1430
viola             974
tuba              972
bass-clarinet     944
cello             930
flute             878
double-bass       852
trombone          831
saxophone         733
contrabassoon     710
english-horn      691
french-horn       652
oboe              596
trumpet           485
guitar            107
mandolin           80
banjo              74
dtype: int64